# Training for critical question

## Setup
First we define some constant values and also install all needed libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



### Installation

In [ ]:
%%capture
import shutil
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import logging
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

### Path Variables and Logger

In [ ]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 1
BASE_MODEL_REPO = "unsloth/Meta-Llama-3.1-8B-Instruct"
MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
NOTEBOOK_PATH = "/content/drive/MyDrive/HSG/NLP/Project NLP/Training/3_Training.ipynb"

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Datasets/SocraticQ/train.csv"

log_base_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Logs/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)

save_notebook_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Training_{TRAINING_NUMBER}.ipynb"

################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [ ]:
logger.info("--------  Start with Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

## Training Parameters

In [ ]:
################################################################################
#######################   Unlsoth Parameters    ################################
################################################################################

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


################################################################################
#######################   PEFT Parameters       ################################
################################################################################

r = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
lora_alpha = 16
lora_dropout = 0 # Supports any, but = 0 is optimized
bias = "none"    # Supports any, but = "none" is optimized
use_gradient_checkpointing = "unsloth" # True or "unsloth" for very long context
random_state = 3407
use_rslora = False  # Unsloth supports rank stabilized LoRA
loftq_config = None # And LoftQ


################################################################################
#######################   SFT Trainer Parameters   #############################
################################################################################

per_device_train_batch_size = 2
gradient_accumulation_steps = 4
warmup_steps = 5
max_steps = 10 # Normaly 60
learning_rate = 2e-4
fp16 = not is_bfloat16_supported()
bf16 = is_bfloat16_supported()
logging_steps = 1
optim = "adamw_8bit"
weight_decay = 0.01
lr_scheduler_type = "linear"
seed = 3407
output_dir = "outputs"
report_to = "tensorboard"
logging_dir = log_base_path

################################################################################
#######################   Log Parameters            ############################
################################################################################

logger.info("Unlsoth Parameters")
logger.info(f"max_seq_length: {max_seq_length}")
logger.info(f"dtype: {dtype}")
logger.info(f"load_in_4bit: {load_in_4bit}")

logger.info("PEFT Parameters")
logger.info(f"r: {r}")
logger.info(f"target_modules: {target_modules}")
logger.info(f"lora_alpha: {lora_alpha}")
logger.info(f"lora_dropout: {lora_dropout}")
logger.info(f"bias: {bias}")
logger.info(f"use_gradient_checkpointing: {use_gradient_checkpointing}")
logger.info(f"random_state: {random_state}")
logger.info(f"use_rslora: {use_rslora}")

logger.info("SFT Trainer Parameters")
logger.info(f"per_device_train_batch_size: {per_device_train_batch_size}")
logger.info(f"gradient_accumulation_steps: {gradient_accumulation_steps}")
logger.info(f"warmup_steps: {warmup_steps}")
logger.info(f"max_steps: {max_steps}")
logger.info(f"learning_rate: {learning_rate}")
logger.info(f"fp16: {fp16}")
logger.info(f"bf16: {bf16}")
logger.info(f"logging_steps: {logging_steps}")
logger.info(f"optim: {optim}")
logger.info(f"weight_decay: {weight_decay}")
logger.info(f"lr_scheduler_type: {lr_scheduler_type}")
logger.info(f"seed: {seed}")
logger.info(f"output_dir: {output_dir}")
logger.info(f"report_to: {report_to}")
logger.info(f"logging_dir: {logging_dir}")

### Unsloth

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_REPO,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    target_modules = target_modules,
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = use_rslora,
    loftq_config = loftq_config,
)

## Load and preprocess dataset
The raw dataset [SocratiQ](https://github.com/NUS-IDS/eacl23_soqg/tree/main) has a label at the begining of the context. We have to remove that and also tokenize the input for the model training.

In [ ]:
dataset = load_dataset('csv', data_files=train_dataset_path,split="train")

In [ ]:
# crop the dataset for testing
#dataset['train'] = dataset['train'].select(range(500))

In [ ]:
# extract the input text and the label
def split_input(example):
    if ':' in example['input']:
        before, after = example['input'].split(':', 1)
        return {
            'label': before.strip(),
            'input_text': after.strip()
        }
    else:
        return {
            'label': example['input'].strip(),
            'input_text': ''
        }

# Apply the map function
dataset = dataset.map(split_input)

In [ ]:
# If you want to see a sample
print(dataset)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You will see a text and you should generate one critical question for this text.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["input_text"]
    outputs      = examples["target"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
# Check the structure of the dataset
print(dataset[0]['text'])

In [ ]:
print(dataset)

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
    )
    return model_inputs

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=2,
    remove_columns=dataset.column_names, # remove all unnecessary
)

In [ ]:
print(tokenized_dataset)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_dataset,
    dataset_text_field = "input_ids",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = warmup_steps,
        max_steps = max_steps,
        learning_rate = learning_rate,
        fp16 = fp16,
        bf16 = bf16,
        logging_steps = logging_steps,
        optim = optim,
        weight_decay = weight_decay,
        lr_scheduler_type = lr_scheduler_type,
        seed = seed,
        output_dir = output_dir,
        report_to = report_to,
        logging_dir = logging_dir,
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

logger.info(f"GPU  Information before Training")
logger.info(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
logger.info(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

logger.info(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
logger.info(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
logger.info(f"Peak reserved memory = {used_memory} GB.")
logger.info(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
logger.info(f"Peak reserved memory % of max memory = {used_percentage} %.")
logger.info(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Save
Save the finetuned model

### Save Lora delta weights

In [ ]:
model.save_pretrained(model_lora_adapter_save_path)  # Local saving
tokenizer.save_pretrained(model_lora_adapter_save_path)
logger.info(f"Saved LoRA adapters to {model_lora_adapter_save_path}")

### Saving merged model

Save merged model

In [ ]:
# Merge to 16bit
if True:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_16bit",)
  logger.info(f"Saved merged model in 16bit to {model_save_path}")

# Merge to 4bit
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_4bit",)
  logger.info(f"Saved merged model in 4bit to {model_save_path}")

# Just LoRA adapters
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "lora",)
  logger.info(f"Saved LoRA adapters to {model_save_path}")

### GGUF / llama.cpp Conversion
Save model in GGUF format

Some supported quant methods (full list on [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer,)
  logger.info(f"Saved merged model in 8bit Q8_0 to {model_save_path}")

# Save to 16bit GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "f16")
  logger.info(f"Saved merged model in 16bit GGUF to {model_save_path}")

# Save to q4_k_m GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "q4_k_m")
  logger.info(f"Saved merged model in q4_k_m GGUF to {model_save_path}")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
# Copy current Notebook to Log Folder
shutil.copy(NOTEBOOK_PATH, save_notebook_path)